# Inspect Data

This example shows how to inspect the data of a loaded test system.

In [4]:
import andes
from andes.utils.paths import get_case

To show all the rows and columns, change the pandas configuration with

In [15]:
import pandas as pd

pd.options.display.max_columns = None
pd.options.display.max_rows = None

Let's load the Kundur's system.

In [6]:
ss = andes.run(get_case('kundur/kundur_full.xlsx'))

Parsing input file </Users/hcui7/repos/andes/cases/kundur/kundur_full.xlsx>
Input file kundur_full.xlsx parsed in 0.4202 second.
-> Power flow calculation with Newton Raphson method:
Power flow initialized.
0: |F(x)| = 14.9283   
1: |F(x)| = 3.60859   
2: |F(x)| = 0.170093  
3: |F(x)| = 0.00203827
4: |F(x)| = 3.76414e-07
Converged in 5 iterations in 0.0138 second.
Report saved to </Users/hcui7/notebooks/examples/kundur_full_out.txt> in 0.0011 second.
-> Single process finished in 0.6818 second.


## Parameters

Parameters from the input file are stored in `cache.df_in` of the model. For `GENROU`, do

In [11]:
ss.GENROU.cache.df_in

,idx,u,name,bus,gen,coi,Sn,Vn,fn,D,...,S12,xd,xd1,xd2,xq1,xq2,Td10,Td20,Tq10,Tq20
uid,,,,,,,,,,,,,,,,,,,,,
0,1,1,GENROU_1,1,1,None,900,20,60,0,...,1,1.8,0.3,0.25,0.55,0.25,8,0.03,0.4,0.05
1,2,1,GENROU_2,2,2,None,900,20,60,0,...,1,1.8,0.3,0.25,0.55,0.25,8,0.03,0.4,0.05
2,3,1,GENROU_3,3,3,None,900,20,60,0,...,1,1.8,0.3,0.25,0.55,0.25,8,0.03,0.4,0.05
3,4,1,GENROU_4,4,4,None,900,20,60,0,...,1,1.8,0.3,0.25,0.55,0.25,8,0.03,0.4,0.05


Parameters will be **converted** to system per-unit base after loading. To inspect the converted parameters, check the `cache.df` parameter.

In [17]:
ss.GENROU.cache.df

,idx,u,name,bus,gen,coi,Sn,Vn,fn,D,M,ra,xl,xq,kp,kw,S10,S12,xd,xd1,xd2,xq1,xq2,Td10,Td20,Tq10,Tq20
uid,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,1,GENROU_1,1,1,None,900,20,60,0,117.00,0,0.006667,0.188889,0,0,0,1,0.2,0.033333,0.027778,0.061111,0.027778,8,0.03,0.4,0.05
1,2,1,GENROU_2,2,2,None,900,20,60,0,117.00,0,0.006667,0.188889,0,0,0,1,0.2,0.033333,0.027778,0.061111,0.027778,8,0.03,0.4,0.05
2,3,1,GENROU_3,3,3,None,900,20,60,0,111.15,0,0.006667,0.188889,0,0,0,1,0.2,0.033333,0.027778,0.061111,0.027778,8,0.03,0.4,0.05
3,4,1,GENROU_4,4,4,None,900,20,60,0,111.15,0,0.006667,0.188889,0,0,0,1,0.2,0.033333,0.027778,0.061111,0.027778,8,0.03,0.4,0.05
